In [270]:
import json
from json_logic import jsonLogic
from json_logic.builtins import BUILTINS, op_var, op_log
from functools import reduce

In [271]:
def courses(data=None, *_ignored):
    return list(filter(None, op_var(data, "courses")))


def course(data=None, code="", number="", *_ignored):
    for course in courses(data):
        if course["code"] == code and course["number"] == number:
            return course
    return None


def units(data=None, *courses):
    flatten_list = lambda *lists: reduce(lambda a, b: a + b, lists, [])
    courses = flatten_list(*courses)
    
    total = 0
    for course in filter(None, courses):
        total += course["units"]
    return total

def consent(data=None, *consenter):
    op_log("consent", consenter)
    return True

ops = {
    **BUILTINS,
    "courses": courses,
    "course": course,
    "units": units,
    "consent": consent,
}

# Examples

## Example 1

In [272]:
or_1 = {
    "cid": "161622",
    "code": "CPSC",
    "number": "530",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": "CPSC 351; or 3 units from CPSC 219, 233 or 235 and 3 units from MATH 271, 273, 315 and 3 units from STAT 205, 213, 321.",
    "antireq": "Credit for CPSC 530 and 630 will not be allowed.",
    "coreq": None,
}

In [273]:
rule_1 = {
    "cid": "161622",
    "prereq": {
        "or": [
            {"course": ["CPSC", "351"]},
            {
                "and": [
                    {
                        ">=": [
                            {
                                "units": [
                                    {"course": ["CPSC", "219"]},
                                    {"course": ["CPSC", "233"]},
                                    {"course": ["CPSC", "235"]},
                                ]
                            },
                            3,
                        ]
                    },
                    {
                        ">=": [
                            {
                                "units": [
                                    {"course": ["MATH", "271"]},
                                    {"course": ["MATH", "273"]},
                                    {"course": ["MATH", "315"]},
                                ]
                            },
                            3,
                        ]
                    },
                    {
                        ">=": [
                            {
                                "units": [
                                    {"course": ["STAT", "205"]},
                                    {"course": ["STAT", "213"]},
                                    {"course": ["STAT", "321"]},
                                ]
                            },
                            3,
                        ]
                    },
                ]
            },
        ]
    },
    "antireq": {
        "or": [
            {"course": ["CPSC", "530"]},
            {"course": ["CPSC", "630"]},
        ]
    },
    "coreq": None,
}

In [274]:
data_1 = {
    "courses": [
        {"code": "CPSC", "number": "329", "gpv": 2.7, "semester": "W2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "526", "gpv": 2.7, "semester": "F2023", "units": 6.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "219", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "MATH", "number": "313", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "MATH", "number": "271", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "STAT", "number": "205", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "351", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": ["ANMEBAH", "SPAN-MIN"],
}

a = jsonLogic(rule_1["prereq"], data_1, ops)
b = jsonLogic(rule_1["antireq"], data_1, ops)

bool(a), bool(b)

(True, False)

## Example 2

In [275]:
or_2 = {
    "cid": "160417",
    "code": "CPSC",
    "number": "219",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": "CPSC 217 or DATA 211.",
    "antireq": "Credit for CPSC 219 and any of 233, 235, ENEL 497 or ENCM 493 will not be allowed.",
    "coreq": None,
}

In [276]:
rule_2 = {
    "cid": "160417",
    "prereq": {
        "or": [
            {"course": ["CPSC", "217"]}, 
            {"course": ["DATA", "211"]}
        ]
    },
    "antireq": {
        "and": [
            {"course": ["CPSC", "219"]},
            {
                "or": [
                    [
                        {"course": ["CPSC", "233"]},
                        {"course": ["CPSC", "235"]},
                        {"course": ["ENEL", "497"]},
                        {"course": ["ENCM", "493"]},
                    ],
                    True
                ]
            },
        ]
    },
    "coreq": None,
}

json.dumps(rule_1)

'{"cid": "161622", "prereq": {"or": [{"course": ["CPSC", "351"]}, {"and": [{">=": [{"units": [{"course": ["CPSC", "219"]}, {"course": ["CPSC", "233"]}, {"course": ["CPSC", "235"]}]}, 3]}, {">=": [{"units": [{"course": ["MATH", "271"]}, {"course": ["MATH", "273"]}, {"course": ["MATH", "315"]}]}, 3]}, {">=": [{"units": [{"course": ["STAT", "205"]}, {"course": ["STAT", "213"]}, {"course": ["STAT", "321"]}]}, 3]}]}]}, "antireq": {"or": [{"course": ["CPSC", "530"]}, {"course": ["CPSC", "630"]}]}, "coreq": null}'

In [277]:
data_2 = {
    "courses": [
        {"code": "CPSC", "number": "217", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "219", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "233", "gpv": 2.7, "semester": "F2023", "units": 6.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": ["ANMEBAH", "SPAN-MIN"],
}

a = jsonLogic(rule_2["prereq"], data_2, operations=ops)
b = jsonLogic(rule_2["antireq"], data_2, operations=ops)

bool(a), bool(b)

(True, True)

## Example 3

In [278]:
or_3 = {
    "cid": "164005",
    "code": "CPSC",
    "number": "405",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": "SENG 300 or 301.",
    "antireq": "Credit for CPSC 405 and ENTI 415 will not be allowed.",
    "coreq": None,
}

In [279]:
rule_3 = {
    "cid": "164005",
    "prereq": {
        "or": [
            {"course": ["SENG", "300"]}, 
            {"course": ["SENG", "301"]}
        ]
    },
    "antireq": {
        "or": [
            {"course": ["CPSC", "405"]},
            {"course": ["ENTI", "415"]},
        ]
    },
    "coreq": None,
}

json.dumps(rule_1)

'{"cid": "161622", "prereq": {"or": [{"course": ["CPSC", "351"]}, {"and": [{">=": [{"units": [{"course": ["CPSC", "219"]}, {"course": ["CPSC", "233"]}, {"course": ["CPSC", "235"]}]}, 3]}, {">=": [{"units": [{"course": ["MATH", "271"]}, {"course": ["MATH", "273"]}, {"course": ["MATH", "315"]}]}, 3]}, {">=": [{"units": [{"course": ["STAT", "205"]}, {"course": ["STAT", "213"]}, {"course": ["STAT", "321"]}]}, 3]}]}]}, "antireq": {"or": [{"course": ["CPSC", "530"]}, {"course": ["CPSC", "630"]}]}, "coreq": null}'

In [280]:
data_3 = {
    "courses": [
        {"code": "SENG", "number": "300", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "405", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": ["ANMEBAH", "SPAN-MIN"],
}

a = jsonLogic(rule_3["prereq"], data_3, operations=ops)
b = jsonLogic(rule_3["antireq"], data_3, operations=ops)


bool(a), bool(b)

(True, True)

## Example 4